In [1]:
import requests
import sys
# from tqdm.asyncio import tqdm
from tqdm import tqdm
import gzip
import csv
import aiohttp
import asyncio
import pandas as pd

total = 2918269#len(pd.read_csv("F:\\_Bounty\\humana_payers.csv"))
filename = "F:\\_Bounty\\humana_payers.csv"

In [2]:
async def get_size(session, semaphore, queue):
    while True:
        url = await queue.get()
        if url is None:
            break


        # if "https" not in url: # for Humana only
        url = "https://storage.googleapis.com/cms-humana-price-transparency-prd/prod/february/inn/" + url.replace("?fileType=innetwork&filename=", "")
        print(url)

        try:
            async with semaphore, session.get(url, ssl=False) as r:
                print(r.status)
                size = r.headers.get("Content-Length")
        except:
            size = None

        await queue.put((url, size))

In [3]:
async def main():
    async with aiohttp.ClientSession(raise_for_status=True) as session:
        semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests

        # Create the queue and fill it with URLs
        queue = asyncio.Queue()
        for chunk in tqdm(pd.read_csv(filename, chunksize=10000), total=total, desc="Chunks"):
            for url in chunk["url"]:
                queue.put_nowait(url)

        # Create a fixed number of worker coroutines to process the URLs
        num_workers = 50
        tasks = []
        for i in range(num_workers):
            tasks.append(asyncio.create_task(get_size(session, semaphore, queue)))

        pbar = tqdm(total = total, desc="Tasks")
        # Wait for all workers to finish processing the URLs
        for task in tasks:
            await task
            pbar.update(1)

        # Extract the results from the queue and add them to the DataFrame
        results = []
        while not queue.empty():
            url, size = await queue.get()
            results.append((url, size))
        df = pd.DataFrame(results, columns=["url", "size"])

        return df

In [4]:
df1 = await main()

Chunks:   0%|          | 292/2918269 [00:05<16:38:29, 48.71it/s]


In [12]:
df1 = df1.sort_values(by="size")

In [13]:
df1.to_csv("F:\\_Bounty\\humana_payers_sorted.csv", index=False)